# Assignment 2.1: Data Lake - Exercise
**Student name: John Kalaiselvan**

## Check Pre-Requisites

In [1]:
%store -r setup_dependencies_passed

In [2]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

In [3]:
print(setup_dependencies_passed)

True


In [4]:
%store -r setup_s3_bucket_passed

In [5]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [6]:
print(setup_s3_bucket_passed)

True


In [7]:
if not setup_dependencies_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Set Source Location

In [15]:
!ls data/

dataset.csv


In [10]:
source_path = "data/dataset.csv"

In [11]:
%store source_path

Stored 'source_path' (str)


## Set S3 Destination Location

In [21]:
s3_dest_path = "s3://{}/homework-2-1/data/".format(bucket)
print(s3_dest_path)

s3://sagemaker-us-east-1-823460696669/homework-2-1/data/


In [22]:
%store s3_dest_path

Stored 's3_dest_path' (str)


## Copy Data From the GitHub to the Private S3 Bucket

In [23]:
!aws s3 cp $source_path $s3_dest_path

upload: data/dataset.csv to s3://sagemaker-us-east-1-823460696669/homework-2-1/data/dataset.csv


In [24]:
!aws s3 ls $s3_dest_path

2026-01-18 15:39:43   20118244 dataset.csv


## Create Athena Database

In [27]:
from pyathena import connect

In [37]:
database_name = "homework21"

In [38]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [39]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [40]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS homework21


In [41]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_40072/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [42]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_40072/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,homework21


## Create Athena Table from Database CSV

### Dataset columns

- `track_id`: A unique ID for the track.
- `artists`: The artists of the track.
- `album_name`: The album that the track belongs to.
- `track_name`: The name of the track.
- `popularity`: The popularity of the track.
- `duration_ms`: Track length.
- `explicit`: Does it haver explicit content?
- `danceability`: Level of danceability between 0-1.
- `energy`: Energy of the track between 0-1.
- `key`: Key of the track.
- `loudness`: Loudness of the track.
- `mode`: Track mode
- `speechiness`: Speech content in the track between 0-1.
- `acousticness`: Acousticness of the track between 0-1.
- `instrumentalness`: Instrumentalness of the track.
- `liveness`: Liveness of the track.
- `valence`: Valence of the track.
- `tempo`: Tempo of the track.
- `time_signature`: Time signature of the track.
- `track_genre`: Track genre.

In [45]:
# Set Athena parameters
table_name_csv = "dataset_csv"

In [48]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [57]:
statement = """DROP TABLE IF EXISTS {}.{}""".format(
    database_name, table_name_csv
)
print(statement)

DROP TABLE IF EXISTS homework21.dataset_csv


In [58]:
pd.read_sql(statement, conn)

/tmp/ipykernel_40072/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [59]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         row_id INT,
         track_id STRING,
         artists STRING,
         album_name STRING,
         track_name STRING,
         popularity INT,
         duration_ms INT,
         explicit BOOLEAN,
         danceability DOUBLE,
         energy DOUBLE,
         key INT,
         loudness DOUBLE,
         mode INT,
         speechiness DOUBLE,
         acousticness DOUBLE,
         instrumentalness DOUBLE,
         liveness DOUBLE,
         valence DOUBLE,
         tempo DOUBLE,
         time_signature INT,
         track_genre STRING
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"',
    'case.insensitive' = 'true'
)
LOCATION '{}'
TBLPROPERTIES (
    'skip.header.line.count'='1',
    'use.null.for.invalid.data'='true'
)""".format(
    database_name, table_name_csv, s3_dest_path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS homework21.dataset_csv(
         row_id INT,
         track_id STRING,
         artists STRING,
         album_name STRING,
         track_name STRING,
         popularity INT,
         duration_ms INT,
         explicit BOOLEAN,
         danceability DOUBLE,
         energy DOUBLE,
         key INT,
         loudness DOUBLE,
         mode INT,
         speechiness DOUBLE,
         acousticness DOUBLE,
         instrumentalness DOUBLE,
         liveness DOUBLE,
         valence DOUBLE,
         tempo DOUBLE,
         time_signature INT,
         track_genre STRING
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"',
    'case.insensitive' = 'true'
)
LOCATION 's3://sagemaker-us-east-1-823460696669/homework-2-1/data/'
TBLPROPERTIES (
    'skip.header.line.count'='1',
    'use.null.for.invalid.data'='true'
)


In [60]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_40072/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


## Query the Data Lake Using SQL

### List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [63]:
statement = """SELECT artists, track_name, popularity FROM {}.{}
WHERE popularity >= 99;""".format(
    database_name, table_name_csv
)
print(statement)

SELECT artists, track_name, popularity FROM homework21.dataset_csv
WHERE popularity >= 99;


In [64]:
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_40072/3103132531.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


                artists                             track_name  popularity
0  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100
1      Bizarrap;Quevedo  Quevedo: Bzrp Music Sessions, Vol. 52          99
2  Sam Smith;Kim Petras              Unholy (feat. Kim Petras)         100


### List artists with an average popularity of 92

In [65]:
statement = """SELECT artists, AVG(popularity) AS avg_popularity FROM {}.{}
GROUP BY artists
HAVING AVG(popularity) = 92""".format(
    database_name, table_name_csv
)
print(statement)

SELECT artists, AVG(popularity) AS avg_popularity FROM homework21.dataset_csv
GROUP BY artists
HAVING AVG(popularity) = 92


In [66]:
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_40072/3103132531.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


             artists  avg_popularity
0       Harry Styles            92.0
1  Rema;Selena Gomez            92.0


### List the Top 10 genres with the highest average energy

In [67]:
statement = """SELECT track_genre, AVG(energy) AS avg_energy FROM {}.{}
GROUP BY track_genre
ORDER BY avg_energy DESC
LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)

SELECT track_genre, AVG(energy) AS avg_energy FROM homework21.dataset_csv
GROUP BY track_genre
ORDER BY avg_energy DESC
LIMIT 10


In [68]:
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_40072/3103132531.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


     track_genre  avg_energy
0    death-metal    0.931470
1      grindcore    0.924201
2      metalcore    0.914485
3          happy    0.910971
4      hardstyle    0.901246
5  drum-and-bass    0.876635
6    black-metal    0.874897
7    heavy-metal    0.874003
8          party    0.871237
9         j-idol    0.868677


### How many tracks is Bad Bunny on?

In [69]:
statement = """SELECT COUNT(*) AS track_count FROM {}.{}
WHERE LOWER(artists) LIKE '%bad bunny%'""".format(
    database_name, table_name_csv
)
print(statement)

SELECT COUNT(*) AS track_count FROM homework21.dataset_csv
WHERE LOWER(artists) LIKE '%bad bunny%'


In [70]:
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_40072/3103132531.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


   track_count
0          416


### The top 10 genres in terms of popularity, sorted by their most popular track

In [71]:
statement = """SELECT track_genre, MAX(popularity) AS max_track_popularity FROM {}.{}
GROUP BY track_genre
ORDER BY max_track_popularity DESC
LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)

SELECT track_genre, MAX(popularity) AS max_track_popularity FROM homework21.dataset_csv
GROUP BY track_genre
ORDER BY max_track_popularity DESC
LIMIT 10


In [72]:
df = pd.read_sql(statement, conn)
print(df)

/tmp/ipykernel_40072/3103132531.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


  track_genre  max_track_popularity
0       dance                   100
1         pop                   100
2     hip-hop                    99
3         edm                    98
4      latino                    98
5      reggae                    98
6   reggaeton                    98
7       latin                    98
8       piano                    96
9        rock                    96
